# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-13-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-13-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-14 05:22:32,33.93911,67.709953,53584,2301,44850,6433.0,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-14 05:22:32,41.15330,20.168300,65334,1256,38860,25218.0,Albania,2270.275905,1.922429
2,NaN,NaN,NaN,Algeria,2021-01-14 05:22:32,28.03390,1.659600,102860,2819,69791,30250.0,Algeria,234.566827,2.740618
3,NaN,NaN,NaN,Andorra,2021-01-14 05:22:32,42.50630,1.521800,8818,87,8070,661.0,Andorra,11412.670679,0.986618
4,NaN,NaN,NaN,Angola,2021-01-14 05:22:32,-11.20270,17.873900,18425,424,15631,2370.0,Angola,56.060518,2.301221


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2237,2244,2244,2253,2257,2264,2277,2288,2301,2301
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1199,1210,1217,1223,1230,1233,1241,1247,1252,1256
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2777,2782,2786,2792,2798,2803,2807,2812,2816,2819


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42530,42666,42666,43291,43440,43740,43948,44137,44608,44850
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,34996,35551,36102,36535,36971,37327,37648,37981,38421,38860
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,67999,68185,68383,68589,68802,69011,69212,69403,69608,69791


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4336,4546,4645,4705,4770,4847,4879,4902,4970,4998
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,14187,14440,14656,14845,15052,15202,15327,15417,15572,15701
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1533,1575,1597,1614,1634,1648,1658,1663,1679,1685


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,50,50,50,50,50,53,54,55,55,55
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,169,169,171,171,171,173,173,173,175,175
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,33,33,35,35,35,35,35,35,35,35


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-14 05:22:32,32.539527,-86.644082,4998,55,0,4943.0,"Autauga, Alabama, US",8945.927079,1.100440
687,1075.0,Lamar,Alabama,US,2021-01-14 05:22:32,33.779950,-88.096680,1157,19,0,1138.0,"Lamar, Alabama, US",8381.021369,1.642178
688,1077.0,Lauderdale,Alabama,US,2021-01-14 05:22:32,34.901719,-87.656247,7350,81,0,7269.0,"Lauderdale, Alabama, US",7926.322941,1.102041


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,23071895,384653,0
India,10512093,151727,10146763
Brazil,8256536,205964,7347080


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,23071895,384653,0,22687242,2021-01-14 05:22:32,37.936303,-91.379001
India,10512093,151727,10146763,213603,2021-01-14 05:22:32,23.088275,81.806127
Brazil,8256536,205964,7347080,703492,2021-01-14 05:22:32,-12.669522,-48.480493
Russia,3434934,62463,2824240,548231,2021-01-14 05:22:32,54.546312,62.120860
United Kingdom,3220953,84910,7430,3128613,2021-01-14 05:22:32,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2837710,31573,0
Texas,2040751,31277,0
Florida,1517472,23396,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2837710,31573,0,2806137,2021-01-14 05:22:32,37.843962,-120.728594
Texas,2040751,31277,0,2009474,2021-01-14 05:22:32,31.660643,-98.653069
Florida,1517472,23396,0,1494076,2021-01-14 05:22:32,28.940755,-82.700744
New York,1179266,40192,0,1139074,2021-01-14 05:22:32,42.544151,-75.474183
Illinois,1046030,19617,0,1026413,2021-01-14 05:22:32,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,958497,12955,0
Illinois,Cook,424217,8870,0
Arizona,Maricopa,397208,6115,0
Florida,Miami-Dade,334217,4472,0
Texas,Harris,269089,3658,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,958497,12955,0,945542,2021-01-14 05:22:32,34.308284,-118.228241,6037.0
Illinois,Cook,424217,8870,0,415347,2021-01-14 05:22:32,41.841448,-87.816588,17031.0
Arizona,Maricopa,397208,6115,0,391093,2021-01-14 05:22:32,33.348359,-112.491815,4013.0
Florida,Miami-Dade,334217,4472,0,329745,2021-01-14 05:22:32,25.611236,-80.551706,12086.0
Texas,Harris,269089,3658,0,265431,2021-01-14 05:22:32,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,53538,63971,102369,8586,18254,176,1730921,162288,28634,382258,...,3127643,26901,77611,1,116983,1515,148171,2105,28596,22297
2021-01-12,53584,64627,102641,8682,18343,176,1744704,162643,28650,383833,...,3173291,27846,77663,1,117299,1520,148968,2107,29757,23239
2021-01-13,53584,65334,102860,8818,18425,176,1757429,163128,28660,385750,...,3220953,28475,77716,1,117811,1521,149769,2109,31100,24256


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,2288,1247,2812,85,420,5,44654,2931,909,6747,...,82096,262,617,0,1073,35,1614,612,471,528
2021-01-12,2301,1252,2816,86,422,5,44848,2941,909,6819,...,83342,269,617,0,1078,35,1630,612,495,551
2021-01-13,2301,1256,2819,87,424,5,44983,2951,909,6868,...,84910,275,618,0,1084,35,1658,612,509,589


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,44137,37981,69403,7724,14825,152,1518715,149873,25843,355530,...,7149,19555,76067,1,110873,1361,132158,1416,20781,13213
2021-01-12,44608,38421,69608,7930,15512,152,1527861,150602,25865,357562,...,7348,20099,76134,1,111175,1361,133599,1416,21074,13396
2021-01-13,44850,38860,69791,8070,15631,152,1536423,151246,25875,359692,...,7430,20733,76197,1,111616,1369,134977,1419,21568,13658


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4336,4546,4645,4705,4770,4847,4879,4902,4970,4998
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1077,1083,1098,1114,1124,1132,1137,1146,1153,1157
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6702,6766,6839,6930,7051,7149,7191,7214,7296,7350
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2128,2149,2181,2217,2263,2315,2324,2340,2351,2368
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,10656,10765,10964,11179,11355,11498,11565,11626,11812,11983


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,404000,50394,0,627541,256344,2758021,362825,213358,65827,49,...,657396,1990204,307483,9078,2143,403386,276686,102282,551942,47395
2021-01-12,407848,50522,0,636100,259553,2804663,364336,217047,66446,49,...,660874,2014645,309629,9247,2166,407947,278544,103203,555249,48072
2021-01-13,410995,50816,0,641729,262020,2837710,366774,220576,67173,49,...,665499,2040751,314218,9368,2166,412545,281202,104392,558020,48289


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-11        4902   15417    1663  2060   5080     957   1637   10537   
2021-01-12        4970   15572    1679  2090   5134     967   1649   10668   
2021-01-13        4998   15701    1685  2109   5170     966   1651   10745   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-11         2651     1545  ...    2302    491     2700      624   
2021-01-12         2697     1568  ...    2314    496     2712      627   
2021-01-13         2734     1569  ...    2346    500     2721      631   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-11           3187  2447  1744          0      817    485  
2021-01-12           3210  2497  1764          0      825    601  
2021-01-13           3222  2502  1774          0      830    601  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,5347,224,0,10147,4081,30480,5213,6416,986,0,...,7865,30585,1396,156,24,5393,3699,1594,5575,489
2021-01-12,5573,225,0,10482,4121,31157,5242,6449,991,0,...,8011,30895,1422,158,24,5477,3789,1634,5633,522
2021-01-13,5760,226,0,10673,4186,31573,5285,6536,994,0,...,8148,31277,1449,158,24,5552,3838,1671,5679,522


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-11          55     173      35   48     77      28     57     178   
2021-01-12          55     175      35   48     79      28     57     195   
2021-01-13          55     175      35   48     80      28     57     197   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-11           66       24  ...      15      9       17        7   
2021-01-12           66       29  ...      16      9       17        7   
2021-01-13           66       29  ...      16      9       17        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-11             24     4     8          0       21      4  
2021-01-12             26     4     9          0       23      4  
2021-01-13             26     4     9          0       23      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.000916,0.005912,0.002203,0.000000,0.003353,0.0,0.005054,0.000968,0.000699,0.004034,...,0.015018,0.027305,0.000503,0.0,0.003199,0.000661,0.005231,0.000475,0.031304,0.038180
2021-01-12,0.000859,0.010255,0.002657,0.011181,0.004876,0.0,0.007963,0.002187,0.000559,0.004120,...,0.014595,0.035129,0.000670,0.0,0.002701,0.003300,0.005379,0.000950,0.040600,0.042248
2021-01-13,0.000000,0.010940,0.002134,0.015665,0.004470,0.0,0.007294,0.002982,0.000349,0.004994,...,0.015020,0.022589,0.000682,0.0,0.004365,0.000658,0.005377,0.000949,0.045132,0.043763


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.004831,0.004835,0.001781,0.000000,0.009615,0.0,0.003573,0.000683,0.0,0.003570,...,0.006485,0.023438,0.000000,NaN,0.011310,0.0,0.006234,0.001637,0.004264,0.041420
2021-01-12,0.005682,0.004010,0.001422,0.011765,0.004762,0.0,0.004345,0.003412,0.0,0.010671,...,0.015177,0.026718,0.000000,NaN,0.004660,0.0,0.009913,0.000000,0.050955,0.043561
2021-01-13,0.000000,0.003195,0.001065,0.011628,0.004739,0.0,0.003010,0.003400,0.0,0.007186,...,0.018814,0.022305,0.001621,NaN,0.005566,0.0,0.017178,0.000000,0.028283,0.068966


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.004301,0.008845,0.002760,0.000000,0.068700,0.0,0.004607,0.003757,0.000232,0.005035,...,0.019974,0.028507,0.000697,0.0,0.006783,0.000000,0.007939,0.006397,0.008884,0.050151
2021-01-12,0.010671,0.011585,0.002954,0.026670,0.046341,0.0,0.006022,0.004864,0.000851,0.005715,...,0.027836,0.027819,0.000881,0.0,0.002724,0.000000,0.010904,0.000000,0.014099,0.013850
2021-01-13,0.005425,0.011426,0.002629,0.017654,0.007671,0.0,0.005604,0.004276,0.000387,0.005957,...,0.011159,0.031544,0.000827,0.0,0.003967,0.005878,0.010314,0.002119,0.023441,0.019558


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,0.005225,0.003505,NaN,0.014542,0.004971,0.014776,0.004644,0.035749,0.008487,0.0,...,0.005394,0.011185,0.004850,0.012379,0.000000,0.011357,0.018745,0.010572,0.003057,0.012022
2021-01-12,0.009525,0.002540,NaN,0.013639,0.012518,0.016911,0.004165,0.017290,0.009403,0.0,...,0.005291,0.012281,0.006979,0.018616,0.010733,0.011307,0.006715,0.009005,0.005992,0.014284
2021-01-13,0.007716,0.005819,NaN,0.008849,0.009505,0.011783,0.006692,0.016259,0.010941,0.0,...,0.006998,0.012958,0.014821,0.013085,0.000000,0.011271,0.009542,0.011521,0.004991,0.004514


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-11      0.004714  0.005872  0.003016  0.004388  0.002764  0.004197   
2021-01-12      0.013872  0.010054  0.009621  0.014563  0.010630  0.010449   
2021-01-13      0.005634  0.008284  0.003574  0.009091  0.007012 -0.001034   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-11      0.003064  0.003811  0.004547  0.005205  ...  0.036004   
2021-01-12      0.007330  0.012432  0.017352  0.014887  ...  0.005213   
2021-01-13      0.001213  0.007218  0.013719  0.000638  ...  0.013829   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-11      0.025052  0.007839  0.006452   0.007906  0.039949  0.021077   
2021-01-12      0.010183  0.004444  0.004808   0.007217  0.020433  0.011468   
2021-01-13      0.008065  0.003319  0.006380   0.003738  0.002002  0.005669   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-11            NaN  0.014907  0.000000  
2021-01-12            NaN  0.009792  0.239175  
2021-01-13            NaN  0.006061  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,0.002437,0.000000,NaN,0.000592,0.009399,0.015865,0.000960,0.014548,0.014403,NaN,...,0.010276,0.003478,0.002874,0.000000,0.0,0.001858,0.000270,0.007585,0.000898,0.000000
2021-01-12,0.042267,0.004464,NaN,0.033015,0.009802,0.022211,0.005563,0.005143,0.005071,NaN,...,0.018563,0.010136,0.018625,0.012821,0.0,0.015576,0.024331,0.025094,0.010404,0.067485
2021-01-13,0.033555,0.004444,NaN,0.018222,0.015773,0.013352,0.008203,0.013490,0.003027,NaN,...,0.017101,0.012364,0.018987,0.000000,0.0,0.013694,0.012932,0.022644,0.008166,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2021-01-11      0.018519  0.000000     0.0  0.0  0.000000     0.0    0.0   
2021-01-12      0.000000  0.011561     0.0  0.0  0.025974     0.0    0.0   
2021-01-13      0.000000  0.000000     0.0  0.0  0.012658     0.0    0.0   

Province_State                               ...   Wyoming                  \
Admin2           Calhoun Chambers  Cherokee  ...      Park Platte Sheridan   
2021-01-11      0.000000      0.0  0.000000  ...  0.000000    0.0      0.0   
2021-01-12      0.095506      0.0  0.208333  ...  0.066667    0.0      0.0   
2021-01-13      0.010256      0.0  0.000000  ...  0.000000    0.0      0.0   

Province_State                                                               
Admin2         Sublette Sweetwater Teton  Uinta Unassigned  Washakie Weston  
2021-01-11          0.0   0.000000   0.0  0.000        NaN  0.000000    0.0  
2021-01-12          0.0   0.083333   0.0  0.125        NaN  0.095238    0.0  
2021-01-13          0.0   0.000000   0.0  0.000        NaN  0.000000    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.001368,0.007854,0.002309,0.002662,0.003466,0.013693,0.005454,0.001534,0.000640,0.004608,...,0.017335,0.032611,0.000561,0.0,0.003451,0.000866,0.005140,0.000276,0.037364,0.042698
2021-01-12,0.001114,0.009054,0.002483,0.006922,0.004171,0.006847,0.006709,0.001861,0.000599,0.004364,...,0.015965,0.033870,0.000616,0.0,0.003076,0.002083,0.005259,0.000613,0.038982,0.042473
2021-01-13,0.000557,0.009997,0.002308,0.011293,0.004321,0.003423,0.007001,0.002421,0.000474,0.004679,...,0.015492,0.028229,0.000649,0.0,0.003721,0.001371,0.005318,0.000781,0.042057,0.043118


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.004583,0.005053,0.001723,0.001489,0.005575,2.328306e-10,0.003067,0.001950,1.680483e-08,0.005232,...,0.008492,0.028565,0.000533,NaN,0.006762,1.460929e-41,0.009988,0.001228,0.014978,0.042577
2021-01-12,0.005132,0.004531,0.001573,0.006627,0.005168,1.164153e-10,0.003706,0.002681,8.402417e-09,0.007952,...,0.011835,0.027641,0.000267,NaN,0.005711,7.304647e-42,0.009950,0.000614,0.032967,0.043069
2021-01-13,0.002566,0.003863,0.001319,0.009127,0.004954,5.820766e-11,0.003358,0.003041,4.201208e-09,0.007569,...,0.015324,0.024973,0.000944,NaN,0.005638,3.652324e-42,0.013564,0.000307,0.030625,0.056017


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-11,0.004977,0.009347,0.002867,0.001030,0.057338,0.002193,0.005140,0.003957,0.000274,0.005760,...,0.021239,0.032120,0.000869,0.0,0.004260,0.000730,0.008507,0.003454,0.008432,0.031756
2021-01-12,0.007824,0.010466,0.002910,0.013850,0.051839,0.001096,0.005581,0.004410,0.000563,0.005738,...,0.024538,0.029970,0.000875,0.0,0.003492,0.000365,0.009705,0.001727,0.011266,0.022803
2021-01-13,0.006625,0.010946,0.002770,0.015752,0.029755,0.000548,0.005593,0.004343,0.000475,0.005847,...,0.017849,0.030757,0.000851,0.0,0.003729,0.003121,0.010010,0.001923,0.017353,0.021181


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,0.007461,0.004785,NaN,0.016404,0.008860,0.016090,0.005971,0.019802,0.010780,8.355904e-92,...,0.008114,0.010463,0.007032,0.016756,0.001227,0.012524,0.012523,0.013425,0.004252,0.007863
2021-01-12,0.008493,0.003662,NaN,0.015022,0.010689,0.016501,0.005068,0.018546,0.010092,4.177952e-92,...,0.006702,0.011372,0.007006,0.017686,0.005980,0.011916,0.009619,0.011215,0.005122,0.011074
2021-01-13,0.008104,0.004741,NaN,0.011935,0.010097,0.014142,0.005880,0.017403,0.010516,2.088976e-92,...,0.006850,0.012165,0.010913,0.015386,0.002990,0.011593,0.009581,0.011368,0.005056,0.007794


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-11      0.008073  0.007959  0.005662  0.007253  0.004522  0.004830   
2021-01-12      0.010972  0.009006  0.007642  0.010908  0.007576  0.007640   
2021-01-13      0.008303  0.008645  0.005608  0.009999  0.007294  0.003303   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-11      0.004784  0.005437  0.007355  0.007115  ...  0.020340   
2021-01-12      0.006057  0.008935  0.012354  0.011001  ...  0.012776   
2021-01-13      0.003635  0.008076  0.013036  0.005819  ...  0.013303   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-11      0.015379  0.005954  0.004339   0.005872  0.023599  0.014962   
2021-01-12      0.012781  0.005199  0.004573   0.006544  0.022016  0.013215   
2021-01-13      0.010423  0.004259  0.005476   0.005141  0.012009  0.009442   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-11           -1.0  0.010045  0.000541  
2021-01-12           -1.0  0.009918  0.119858  
2021-01-13           -1.0  0.007989  0.059929  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-11,0.007746,0.001232,NaN,0.006843,0.009398,0.015430,0.003670,0.008118,0.010255,NaN,...,0.011167,0.005878,0.004608,0.001762,0.000170,0.003597,0.001919,0.010020,0.002636,0.002170
2021-01-12,0.025006,0.002848,NaN,0.019929,0.009600,0.018821,0.004617,0.006631,0.007663,NaN,...,0.014865,0.008007,0.011616,0.007291,0.000085,0.009587,0.013125,0.017557,0.006520,0.034827
2021-01-13,0.029280,0.003646,NaN,0.019075,0.012686,0.016086,0.006410,0.010061,0.005345,NaN,...,0.015983,0.010186,0.015302,0.003646,0.000042,0.011640,0.013028,0.020100,0.007343,0.017414


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-11      0.021500  0.001678  0.000962  0.000691  0.009272  0.017070   
2021-01-12      0.010750  0.006619  0.000481  0.000345  0.017623  0.008535   
2021-01-13      0.005375  0.003310  0.000241  0.000173  0.015141  0.004267   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-11      0.033895  0.022461  0.011911 -0.000375  ...  0.012250   
2021-01-12      0.016947  0.058983  0.005955  0.103979  ...  0.039458   
2021-01-13      0.008474  0.034620  0.002978  0.051990  ...  0.019729   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-11      0.000977  0.000017  0.001351   0.009705  0.000097  0.004464   
2021-01-12      0.000488  0.000009  0.000675   0.046519  0.000048  0.064732   
2021-01-13      0.000244  0.000004  0.000338   0.023260  0.000024  0.032366   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-11      -0.501961  0.000878  0.014323  
2021-01-12      -0.501961  0.048058  0.007161  
2021-01-13      -0.501961  0.024029  0.003581  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210113,AK,49203,NaN,1321632,NaN,totalTestsViral,1370835,68.0,1138.0,...,1370835,2,34,c539733f6fd3701c417790f59c2259ce82b2ec76,0,0,0,0,0,NaN
1,20210113,AL,410995,83867.0,1662596,NaN,totalTestsPeopleViral,1989724,2975.0,38229.0,...,2073591,187,642,1d4d038c95419e0dfbe173e4d192871cebe7050c,0,0,0,0,0,NaN
2,20210113,AR,259553,48680.0,1997863,NaN,totalTestsViral,2208736,1354.0,12414.0,...,2257416,0,101,e5831236f6e1ef7c2dbfc0dc6438d3bce8d6caf4,0,0,0,0,0,NaN
3,20210113,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,79decf68223011f7ae4b86f10f346c2b044c146f,0,0,0,0,0,NaN
4,20210113,AZ,641729,34881.0,2524887,NaN,totalTestsViral,5788321,5055.0,44863.0,...,3166616,191,474,6bf8717a62a21e9970b38349dde3c1daffc56210,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210113,AK,49203.0,NaN,1321632.0,NaN,totalTestsViral,1370835.0,68.0,1138.0,...,1370835,2,34,c539733f6fd3701c417790f59c2259ce82b2ec76,0,0,0,0,0,NaN
1,20210113,AL,410995.0,83867.0,1662596.0,NaN,totalTestsPeopleViral,1989724.0,2975.0,38229.0,...,2073591,187,642,1d4d038c95419e0dfbe173e4d192871cebe7050c,0,0,0,0,0,NaN
2,20210113,AR,259553.0,48680.0,1997863.0,NaN,totalTestsViral,2208736.0,1354.0,12414.0,...,2257416,0,101,e5831236f6e1ef7c2dbfc0dc6438d3bce8d6caf4,0,0,0,0,0,NaN
3,20210113,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,79decf68223011f7ae4b86f10f346c2b044c146f,0,0,0,0,0,NaN
4,20210113,AZ,641729.0,34881.0,2524887.0,NaN,totalTestsViral,5788321.0,5055.0,44863.0,...,3166616,191,474,6bf8717a62a21e9970b38349dde3c1daffc56210,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-13,AK,49203,NaN,1321632,NaN,totalTestsViral,1370835,68.0,1138.0,NaN,...,1370835,2,34,c539733f6fd3701c417790f59c2259ce82b2ec76,0,0,0,0,0,NaN
2021-01-13,AL,410995,83867.0,1662596,NaN,totalTestsPeopleViral,1989724,2975.0,38229.0,NaN,...,2073591,187,642,1d4d038c95419e0dfbe173e4d192871cebe7050c,0,0,0,0,0,NaN
2021-01-13,AR,259553,48680.0,1997863,NaN,totalTestsViral,2208736,1354.0,12414.0,452.0,...,2257416,0,101,e5831236f6e1ef7c2dbfc0dc6438d3bce8d6caf4,0,0,0,0,0,NaN
2021-01-13,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,79decf68223011f7ae4b86f10f346c2b044c146f,0,0,0,0,0,NaN
2021-01-13,AZ,641729,34881.0,2524887,NaN,totalTestsViral,5788321,5055.0,44863.0,1158.0,...,3166616,191,474,6bf8717a62a21e9970b38349dde3c1daffc56210,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-13,AK,49203.0,NaN,1321632.0,NaN,totalTestsViral,1370835.0,68.0,1138.0,NaN,...,1370835,2,34,c539733f6fd3701c417790f59c2259ce82b2ec76,0,0,0,0,0,NaN
2021-01-13,AL,410995.0,83867.0,1662596.0,NaN,totalTestsPeopleViral,1989724.0,2975.0,38229.0,NaN,...,2073591,187,642,1d4d038c95419e0dfbe173e4d192871cebe7050c,0,0,0,0,0,NaN
2021-01-13,AR,259553.0,48680.0,1997863.0,NaN,totalTestsViral,2208736.0,1354.0,12414.0,452.0,...,2257416,0,101,e5831236f6e1ef7c2dbfc0dc6438d3bce8d6caf4,0,0,0,0,0,NaN
2021-01-13,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,79decf68223011f7ae4b86f10f346c2b044c146f,0,0,0,0,0,NaN
2021-01-13,AZ,641729.0,34881.0,2524887.0,NaN,totalTestsViral,5788321.0,5055.0,44863.0,1158.0,...,3166616,191,474,6bf8717a62a21e9970b38349dde3c1daffc56210,0,0,0,0,0,NaN
